In [1]:
using ForwardDiff, LinearAlgebra, Random, Statistics
const FD = ForwardDiff;

In [2]:
relu(x) = x>0 ? x : 0

relu (generic function with 1 method)

In [3]:
nnLayer(x,W,b,σ) = σ.(reshape(W,(size(b,1),:))*x .+ b)

function getRandomParams(layerStructure)
    nParams = sum(X[1][1]*X[1][2]+X[1][2] for X in layerStructure);
    return randn(nParams)
end

function neuralNet(x,layerStructure,params)
    X = x;
    n = 1;
    for (layer, σ) in layerStructure
        X = nnLayer(X,params[n:(n-1+layer[1]*layer[2])],params[(n+layer[1]*layer[2]) : (n-1+layer[1]*layer[2]+layer[2])],σ)
        n += layer[1]*layer[2]+layer[2];
    end
    return X
end

layerStructure = [
    (2 => 40,tanh),
    (40 => 40,tanh),
    # (40 => 40,tanh),
    # (40 => 40,tanh),
    # (40 => 40,tanh),
    (40 => 2,identity) # Re, Im
];

ψmodel(xt,params) = neuralNet(xt,layerStructure,params);
initParams = getRandomParams(layerStructure);

In [5]:
layerStructure |> typeof

Vector{Tuple{Pair{Int64, Int64}, Function}} (alias for Array{Tuple{Pair{Int64, Int64}, Function}, 1})

In [484]:
test = randn(2,10)

2×10 Matrix{Float64}:
 0.0268331  0.0645446  0.222067   0.182439  …  0.938917  0.85872  -0.547609
 0.187715   1.27266    1.12085   -3.42391      0.923609  0.916    -0.727696

In [485]:
hcat([ψmodel(X,initParams) for X in eachcol(test)]...) .- ψmodel(test,initParams)

2×10 Matrix{Float64}:
 3.55271e-15   1.77636e-15  -3.55271e-15  …   1.77636e-15  -8.88178e-16
 1.77636e-15  -1.77636e-15   0.0             -1.77636e-15   3.55271e-15

In [486]:
using Zygote

In [487]:
ψmodel(randn(2,10),initParams)


2×10 Matrix{Float64}:
 -2.24123  5.51409  -8.94528  -6.82338  …  -7.23743   7.10879  4.48273
  3.00394  3.21835  -6.26823  -1.47565     -3.11407  -7.9877   2.24781

Let $E$ be some units of energy. The dimension-less Schrodinger equation is:
$$
i \frac{\partial \psi}{\partial \tau} = -\frac{1}{2} \frac{\partial^2\psi}{\partial \xi^2} + (V(\beta\xi)/E)\psi
\\
\beta = \frac{\hbar}{\sqrt{m E}}
\\
x = \beta \xi
\\
t = \frac{\hbar}{E} \tau
$$

In [489]:
ψmodel((transpose∘hcat)(collocationPts[2,:],collocationPts[1,:]),initParams)

MethodError: MethodError: no method matching +(::Transpose{Float64, Vector{Float64}}, ::Float64)
For element-wise addition, use broadcasting with dot syntax: array .+ scalar
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:591
  +(!Matched::T, ::T) where T<:Union{Float16, Float32, Float64} at float.jl:383
  +(!Matched::Base.TwicePrecision, ::Number) at twiceprecision.jl:290
  ...

In [490]:
test = FD.jacobian((X -> (ψmodel((transpose∘hcat)(X, collocationPts[2,:]),initParams))),collocationPts[1,:])

MethodError: MethodError: no method matching one(::Type{SubArray{Float64, 1, Matrix{Float64}, Tuple{Int64, Base.Slice{Base.OneTo{Int64}}}, true}})
Closest candidates are:
  one(!Matched::Union{Type{T}, T}) where T<:AbstractString at strings/basic.jl:262
  one(!Matched::Union{Type{P}, P}) where P<:Dates.Period at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/periods.jl:54
  one(!Matched::Bidiagonal{T, V} where V<:AbstractVector{T}) where T at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/special.jl:373
  ...

In [491]:
using DiffResults

In [492]:
nCollocationPts

50

In [413]:
∂ₓψ(ψ,xt) = FD.jacobian((X -> (ψ((transpose∘hcat)(X, xt[2,:])))),xt[1,:]) |> X -> hcat([X[2i-1:2i,i] for i in (eachindex∘eachcol)(xt)]...);
∂ₜψ(ψ,xt) = FD.jacobian((T -> (ψ((transpose∘hcat)(xt[1,:],T)))),xt[2,:]) |> X -> hcat([X[2i-1:2i,i] for i in (eachindex∘eachcol)(xt)]...);

In [525]:
# Dψ(ψ,xt) = FD.jacobian(ψ,xt);
# ∂ₜψ(ψ,x,t) = Zygote.jacobian(t -> ψ(x,t),t)[1];
∂ₜψ(ψ,x,t) = Zygote.forward_jacobian(t -> ψ(x,t),t)[2]';
# ∂ₜψ(ψ,x,t) = FD.derivative(t -> ψ(x,t),t);
# ∂ₓ²ψ(ψ,x,t) = FD.derivative(x -> FD.derivative(x -> ψ(x,t),x),x);
∂ₓ²ψ(ψ,x,t) = Zygote.forward_jacobian(x -> Zygote.forward_jacobian(x -> ψ(x,t),x)[2],x)[2]';
# ∂ₓ²ψ(ψ,x,t) = Zygote.jacobian(x -> Zygote.jacobian(x -> ψ(x,t),x)[2],x)[1];
TDSE(ψ,V,x,t) = J*∂ₜψ(ψ,x,t) - (-∂ₓ²ψ(ψ,x,t)/2) #+ V(x)*ψ(x,t))

TDSE (generic function with 1 method)

In [526]:
TDSE(ψtest,V,randn(2)...)
# ∂ₓ²ψ(ψtest,randn(2)...)

2×1 Matrix{Float64}:
 1.7763568394002505e-15
 0.0

In [527]:
xmin = 0; xmax = 1; Tmax = 50;
V(x) = 0;
nCollocationPts = 1000;
# collocationPts = (collect∘transpose∘hcat)(rand(nCollocationPts),Tmax*rand(nCollocationPts))
collocationPts = (collect∘eachrow∘hcat)(rand(nCollocationPts),Tmax*rand(nCollocationPts))

1000-element Vector{SubArray{Float64, 1, Matrix{Float64}, Tuple{Int64, Base.Slice{Base.OneTo{Int64}}}, true}}:
 [0.21383136651668588, 41.97353432168335]
 [0.8472980084181388, 15.365819014968402]
 [0.9919786709722269, 16.085670045777334]
 [0.17729096186680848, 24.79823889575552]
 [0.5034830181661122, 16.00049904697713]
 [0.8469995441222546, 41.41320667707322]
 [0.46822000826661425, 31.994696603132045]
 [0.41358679169598445, 21.91606388888235]
 [0.4849976530621155, 45.23655716814544]
 [0.6911793367297117, 46.04569697940715]
 ⋮
 [0.8839822960983407, 1.8914453703719303]
 [0.03622021170238965, 36.12135840741045]
 [0.10279070165454907, 13.849325905606696]
 [0.46899409086329835, 43.96809938099744]
 [0.24624997515800007, 31.32346129569671]
 [0.049918267515767334, 28.05355140204809]
 [0.44008342239674714, 37.17145725542595]
 [0.5050443677791251, 7.950988098637407]
 [0.1291197335960238, 28.173141071821213]

In [534]:


function test_loss(model, params)
    # ψNN(x,t) = model([x,t],params);
    return sum(TDSE((x,t) -> model([x,t],params),V,collocationPts[1]...))
    # sum([norm(TDSE(ψNN,V,X...)) for X in collocationPts])
end

test_loss (generic function with 1 method)

In [535]:
# using Zygote

test_loss(ψmodel,initParams)
# test_loss(ψmodel,initParams)

-0.038446830110090965

In [536]:
Zygote.jacobian(p->test_loss(ψmodel, p),initParams)
# FD.gradient(p->test_loss(ψmodel, p),initParams)

ErrorException: Mutating arrays is not supported -- called setindex!(Matrix{Float64}, ...)
This error occurs when you ask Zygote to differentiate operations that change
the elements of arrays in place (e.g. setting values with x .= ...)

Possible fixes:
- avoid mutating operations (preferred)
- or read the documentation and solutions for this error
  https://fluxml.ai/Zygote.jl/latest/limitations


In [113]:
ψtest(x,t) = (collect∘reim)( sin(π*x)*exp(-1.0im*π^2*t/2) + sin(2π*x)*exp(-1.0im*π^2*2^2*t/2));
TDSE(ψtest,x->0,randn(2)...)

2-element Vector{Float64}:
 0.0
 8.881784197001252e-16

In [103]:
let xt=randn(2), V(x) = 0
    norm(J*∂ₜψ(ψtest,xt...) - (-∂ₓ²ψ(ψtest,xt...)/2 + V(xt[1])*ψtest(xt...)))
end

1.831026719408895e-15

In [92]:
[0 -1; 1 0]*[1,0]

2-element Vector{Int64}:
 0
 1